In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
df=pd.read_csv("/kaggle/input/arabic-classification/arabic_dataset_classifiction.csv/arabic_dataset_classifiction.csv")

In [3]:
df

,text,targe
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0
...,...,...
111723,اللاعب تأخر في العودة إلى التداريب والمدرب غاض...,4
111724,المشرف العام لحسنية أكادير قال إنه سيغادر الفر...,4
111725,نسب إليه نتائج الوداد وصحوة الرجاء وآخر صيحاته...,4
111726,ستحتضن الرباط في الفترة مابين يوليوز المقبل دو...,4


In [4]:
df.isna().sum()

text     2939
targe       0
dtype: int64

In [5]:
df = df.dropna(subset=["text"])
df = df[df["text"].str.strip() != ""]

# more preprocessing

In [6]:
import re

ARABIC_DIACRITICS = re.compile(r"""
    ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ
""", re.VERBOSE)

def normalize_arabic(text):
    text = re.sub(ARABIC_DIACRITICS, '', text)
    text = re.sub(r'[إأآٱا]', 'ا', text)
    text = re.sub(r'[ىي]', 'ي', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'[گڤپچ]', 'ك', text)
    return text


def remove_noise(text):
    text = re.sub(r'http\S+|www\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'[A-Za-z]', ' ', text)
    text = re.sub(r'[0-9٠-٩]', ' ', text)
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)
    return text


def reduce_elongation(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)


def clean_spacing(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def clean_arabic_text_v2(text):
    text = normalize_arabic(text)
    text = remove_noise(text)
    text = reduce_elongation(text)
    text = clean_spacing(text)
    return text




In [7]:
df["text_clean"] = df["text"].apply(clean_arabic_text_v2)

In [8]:
df = df.rename(columns={"targe": "label"})


In [9]:
df=df.iloc[:25000]

In [10]:
df

,text,label,text_clean
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0,بين استوديوهات ورزازات وصحراء مرزوكه واثار ولي...
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0,قررت النجمه الامريكيه اوبرا وينفري الا يقتصر ع...
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0,اخبارنا المغربيه الوزاني تصوير الشملالي الهب ا...
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0,اخبارنا المغربيه قال ابراهيم الراشدي محامي سعد...
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0,تزال صناعه الجلود في المغرب تتبع الطريقه التقل...
...,...,...,...
24997,فتحت المصلحة الولائية للشرطة القضائية بمدينة ا...,1,فتحت المصلحه الولاييه للشرطه القضاييه بمدينه ا...
24998,شب حريق مهول في الساعات الأولى من صباح اليوم ا...,1,شب حريق مهول في الساعات الاولي من صباح اليوم ا...
24999,أسفرت العمليات الأمنية التي باشرتها مصالح ولاي...,1,اسفرت العمليات الامنيه التي باشرتها مصالح ولاي...
25000,تمكنت عناصر الجمارك بفاس من حجز َّ كيلوغراما م...,1,تمكنت عناصر الجمارك بفاس من حجز كيلوغراما من م...


In [11]:
print(df["text"].iloc[5])
print(df["text_clean"].iloc[5])

مصطفى الصوفي انطلقت أمس فعاليات الدورة الثالثة من مهرجان سلا الدولي للفنون الساخرة التي تنظمها جمعية نجمة للفكر والإبداع والتي تستمر على مدى ثلاثة أيام تحت شعار نضحكوها مغربية وشهد حفل الافتتاح الذي عرف تقديم كلمة مهمة لرئيسة الجمعية فوزية الوافي تكريم الثنائي سفاج ومهيول هذا الثنائي الذي اعتبرته مديرة المهرجان زهور الزرييق من الثنائيات الكوميدية الناجحة والرائعة التي قدمت طيلة مسيرتها الفنية فيضا من الأعمال التي خلقت الفرجة الكوميدية للجمهور وبالرغم من مرض واحد من الثنائي والمقبل على عملية جراحية حيث نتمنى له الشفاء العاجل لا أن الثنائي المحبوب حل إلى المركب الثقافي الملكي من اجل لقاء الجمهور لقاء تميز بالبساطة والود وشهادة مؤثرة ومميزة قدمها الفنان الساخر الشرقي الساروتي في حق المحتفى بهما واللذين تلقيا هدايا وتذكارات من يد رئيسة الجمعية ورئيسة الجماعة الحضرية كما تميزت الدورة أيضا بتكريم الفنان عبد الخالق فيهد فضلا عن سلسة من الفقرات الكوميدية التي سيؤديها نجوم الكوميديا من الجيل الصاعد الفائزين في المباراة الخاصة بانتقاء أحسن الكوميديين الشباب من بروع الوطن ومن مدينة سلا وتشكل هذه 

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_VOCAB = 20000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(df["text_clean"])

sequences = tokenizer.texts_to_sequences(df["text_clean"])
X = pad_sequences(sequences, maxlen=MAX_LEN, padding="post")
y = df["label"].values


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_VOCAB = 20_000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(df["text_clean"])

sequences = tokenizer.texts_to_sequences(df["text_clean"])
X = pad_sequences(sequences, maxlen=MAX_LEN, padding="post")
y = df["label"].values

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SimpleRNN, GRU, LSTM

EMBED_DIM = 128  # can be smaller than Word2Vec, faster
NUM_CLASSES = len(set(y))

def build_model(cell_type):
    inputs = Input(shape=(MAX_LEN,))
    
    # trainable embeddings
    x = Embedding(
        input_dim=MAX_VOCAB,
        output_dim=EMBED_DIM,
        trainable=True  # embedding weights will be learned
    )(inputs)
    
    if cell_type == "rnn":
        x = SimpleRNN(128)(x)
    elif cell_type == "gru":
        x = GRU(128)(x)
    elif cell_type == "lstm":
        x = LSTM(128)(x)
    
    x = Dropout(0.3)(x)
    outputs = Dense(NUM_CLASSES, activation="softmax")(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

results = {}

for name in ["rnn", "gru", "lstm"]:
    model = build_model(name)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=32,
        verbose=1
    )
    results[name] = history.history


for model_name, history in results.items():
    best_acc = max(history["val_accuracy"])
    print(f"{model_name.upper()} best val accuracy: {best_acc:.4f}")


I0000 00:00:1770055265.950914     254 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/10


I0000 00:00:1770055268.811289     396 service.cc:152] XLA service 0x7bd5b000b990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770055268.811325     396 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1770055269.105218     396 cuda_dnn.cc:529] Loaded cuDNN version 91002


 18/704 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.5495 - loss: 0.6913

I0000 00:00:1770055270.374823     396 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8484 - loss: 0.3312 - val_accuracy: 0.9272 - val_loss: 0.1985
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9317 - loss: 0.1695 - val_accuracy: 0.8960 - val_loss: 0.2778
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9357 - loss: 0.1687 - val_accuracy: 0.8924 - val_loss: 0.2806
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9480 - loss: 0.1284 - val_accuracy: 0.8584 - val_loss: 0.3633
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9671 - loss: 0.0948 - val_accuracy: 0.8760 - val_loss: 0.3673
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9814 - loss: 0.0552 - val_accuracy: 0.8808 - val_loss: 0.3901
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9811 - loss: 0.0586 - val_accuracy: 0.9000 - val_loss: 0.3811
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9800 - loss: 0.0589 - val_accuracy: 0.85

In [17]:
for model_name, history in results.items():
    best_acc = max(history["val_accuracy"])
    print(f"{model_name.upper()} best val accuracy: {best_acc:.4f}")

RNN best val accuracy: 0.9272
GRU best val accuracy: 0.9916
LSTM best val accuracy: 0.9884
